# 第五节 – 多代理协调器

演示使用 Foundry Local 的一个简单的双代理流程（研究员 -> 编辑）。


### 说明：依赖安装
安装 `foundry-local-sdk` 和 `openai`，用于本地模型访问和聊天完成操作。幂等操作。


# 场景
实现一个最小化的双代理编排模式：
- **研究员代理**收集简洁的事实要点
- **编辑代理**重写内容以适应高管的清晰需求

展示每个代理的共享内存、按顺序传递中间输出以及一个简单的管道功能。可扩展至更多角色（例如评论员、验证员）或并行分支。

**环境变量：**
- `FOUNDRY_LOCAL_ALIAS` - 默认使用的模型（默认值：phi-4-mini）
- `AGENT_MODEL_PRIMARY` - 主代理模型（覆盖 ALIAS）
- `AGENT_MODEL_EDITOR` - 编辑代理模型（默认为主模型）

**SDK参考：** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**工作原理：**
1. **FoundryLocalManager** 自动启动 Foundry Local 服务
2. 下载并加载指定的模型（或使用缓存版本）
3. 提供一个兼容 OpenAI 的交互端点
4. 每个代理可以使用不同的模型来完成专门任务
5. 内置重试逻辑优雅地处理临时故障

**主要功能：**
- ✅ 自动服务发现和初始化
- ✅ 模型生命周期管理（下载、缓存、加载）
- ✅ OpenAI SDK 兼容性，提供熟悉的 API
- ✅ 多模型支持，用于代理任务专精
- ✅ 强大的错误处理和重试逻辑
- ✅ 本地推理（无需云 API）


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### 说明：核心导入与类型定义
引入数据类用于存储代理消息，并使用类型提示提高代码清晰度。导入 Foundry Local 管理器和 OpenAI 客户端，以支持后续代理操作。


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### 说明：模型初始化（SDK模式）
使用 Foundry Local Python SDK 进行稳健的模型管理：
- **FoundryLocalManager(alias)** - 自动启动服务并通过别名加载模型
- **get_model_info(alias)** - 将别名解析为具体的模型ID
- **manager.endpoint** - 为 OpenAI 客户端提供服务端点
- **manager.api_key** - 提供 API 密钥（本地使用时可选）
- 支持为不同代理（主代理与编辑代理）使用独立模型
- 内置带有指数回退的重试逻辑以增强可靠性
- 连接验证以确保服务已准备就绪

**关键 SDK 模式：**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**生命周期管理：**
- 管理器全局存储以确保正确清理
- 每个代理可以使用不同的模型以实现专业化
- 自动服务发现与连接处理
- 在失败时进行带有指数回退的优雅重试

这确保在代理编排开始之前完成正确的初始化。

**参考资料：** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### 说明：Agent 和 Memory 类
定义了轻量级的 `AgentMsg` 用于存储记忆条目，以及封装的 `Agent`，包括：
- **系统角色** - Agent 的角色和指令
- **消息历史** - 维护对话上下文
- **act() 方法** - 执行操作并进行适当的错误处理

Agent 可以使用不同的模型（主模型与编辑器模型），并为每个 Agent 保持独立的上下文。这种模式实现了以下功能：
- 操作间的记忆持久性
- 每个 Agent 的灵活模型分配
- 错误隔离与恢复
- 简单的链式调用和编排


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### 说明：协调管道
创建两个专门的代理：
- **研究员**：使用主模型，收集事实信息
- **编辑员**：可以使用单独的模型（如果已配置），进行优化和重写

`pipeline` 函数：
1. 研究员收集原始信息
2. 编辑员将信息优化为适合决策的输出
3. 返回中间结果和最终结果

这种模式的优势包括：
- 模型专用化（不同角色使用不同模型）
- 通过多阶段处理提升质量
- 信息转化过程的可追溯性
- 易于扩展到更多代理或并行处理


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### 说明：管道执行与结果
针对一个以合规性和延迟为主题的问题执行多代理管道，以展示：
- 信息的多阶段转化
- 代理的专业化与协作
- 通过优化提升输出质量
- 可追溯性（保留中间和最终输出）

**结果结构：**
- `question` - 用户的原始查询
- `research` - 原始研究输出（事实性要点）
- `final` - 精炼后的执行摘要
- `models` - 每个阶段使用的模型

**扩展思路：**
1. 添加一个评论代理进行质量审查
2. 为不同方面实施并行研究代理
3. 添加一个验证代理进行事实核查
4. 为不同复杂度级别使用不同模型
5. 实施反馈循环以进行迭代优化


### 高级：自定义代理配置

尝试通过修改环境变量来自定义代理行为，然后运行初始化单元格：

**可用模型：**
- 在终端中使用 `foundry model ls` 查看所有可用模型
- 示例：phi-4-mini、phi-3.5-mini、qwen2.5-7b、llama-3.2-3b 等。


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。应以原始语言的文档作为权威来源。对于关键信息，建议使用专业人工翻译。我们对因使用此翻译而引起的任何误解或误读不承担责任。
